## Statistical NLP - Assignment 01

Group members: Ilaria Salvatori, Vali Florinel Craciun 

### Task 1

Download the file corpus.zip from LernraumPlus. The file corpus.txt contains the corpus we will use in this
worksheet, there is exactly one sentence in each line of this file. A sentence is a sequence $w_1, . . . , w_N$ of words, where
$w_1$ is the first word in the sentence, $w_N$ is the last word and $N$ is the number of words in the sentence. Now we define
some distributions of words for the provided corpus:
- $P (w)$ is the distribution of all words in the corpus
- $P (w_i|w_{i−1})$ is the distribution of words given the previous word in a word sequence is $w_{i−1}$
- $P (w_i|w_{i−1}, w_{i−2})$ is the distribution of words at position i in a word sequence given the word at position $i-1$ is
$w_{i−1}$ and the word at position $i −2$ is $w_{i−2}$

**Hint** Introduce special words to model the beginning and the end of a sentence!


a) First of all you need to tokenize the corpus. Therefore, implement a Python function `tokenize_sentence()`
which takes a single string as input (representing a sentence) and returns a list of words. You may ignore commas,
semicolons and colons. (*3 points*)

In [1]:
import re

In [32]:
def tokenize_sentence(doc):
    words=[]
    to_ignore=[',', ':', ';' ] # chars to ignore
    

    with open(doc,'r') as f:
        for line in f: 
            for word in line.split(): # each line is splitted 
                if word not in to_ignore:
                    
                    if word[-1]=='.' :  # we want to separate the words with a final point 
                        wordPre=word[:-1]
                        
                        if len(wordPre)>1: # avoid index out of range error in case word == '.'
                        
                            if wordPre[-1]=='.' :  # we also check for duplicate punctuation 
                                wordPre = wordPre[:-1]
                        
                        wordPost=word[-1] # get the last char of the current word 
                        # append both parts 
                        words.append(wordPre)
                        words.append(wordPost)
                    
                    elif word[-1]==',':
                        wordPre=word[0:-1]
                        if wordPre[-1]==',':
                            wordPre = wordPre[0:-1]

                        words.append(wordPre)
                    
                    elif word[-1]==';':
                        wordPre=word[0:-1]
                        if wordPre[-1]==';':
                            wordPre = wordPre[0:-1]
                        words.append(wordPre)

                    elif word[-1]==':':
                        wordPre=word[0:-1]
                        if wordPre[-1]==':':
                            wordPre = wordPre[0:-1]
                        words.append(wordPre)
                    else:
                        words.append(word)

    return words 





In [ ]:
words= tokenize_sentence('corpus.txt') # the output is a list with all words 


b) Provide Python code for representing and learning the distributions $P (w)$, $P (w_i|w_{i−1})$ and $P (w_i|w_{i−1}, w_{i−2})$.
To do this, implement the following functions which should return a probability distribution over the whole
vocabulary:
-  `unigram_distribution()`
- `bigram_distribution(w1)`
- `trigram_distribution(w1, w2)`

(*6 points*)

In [34]:

def unigram_distribution(words):
    total_words = len(words)
    probs = {}
    
    # for each word we get the probability by adding to the dictionary 1/total_words every time we encounter the word
    for word in words:
        if word in probs:
            probs[word] += 1 / total_words
        else:
            probs[word] = 1 / total_words
    
    return probs


def bigram_distribution(w1, words,probs):
    # we consider w1 as wi-1 of the slides 
    cond_probs={}
    indices = [i for i, x in enumerate(words) if x == w1] # indices of w1
    co_occ = [(words[i],words[i+1]) for i in indices] # co occurence list. This is a list with all the tuples (wi-1,wi)
    
    for el in co_occ:
        w1_freq = probs[el[0]]*len(words) # frequency of wi-1
        
        #same process as before but now we want f(wi,wi-1)/f(wi-1)
        # so every time we observe a certain (wi-1,wi) we add 1/f(wi-1)
        if el in cond_probs:
            cond_probs[el] += 1 / w1_freq 
        else:
            cond_probs[el] = 1 / w1_freq


    return cond_probs 


def trigram_distribution(w1,w2, words, probs):   
    # w2 is wi-2
    
    probs=unigram_distribution(words)
    cond_probs={}
    
    indices_w2 = [i for i, x in enumerate(words) if x == w2] # get the indices of wi-2 
    co_occ=[(words[i], words[i+1], words[i+2]) for i in indices_w2 if words[i+1] == w1] # get the triple (wi-2, wi-1, wi) iff the word after w2 is the specified w1
    for el in co_occ:
        p=bigram_distribution(w2,words, probs)
        w1_w2_freq = p[(w2,w1)]*probs[w2]*len(words) #since bigram returns conditional probabilities 
                                                     #we need to multiply them by the p(w2) from the unigram, and then multiply everything by len (words) to get f(w2,w1)
        
        # do the same probability update as before
        if el in cond_probs:
            cond_probs[el] += 1 / w1_w2_freq
        else:
            cond_probs[el] = 1 / w1_w2_freq


    return cond_probs

    

c) How does the number of parameters of these distributions scale with the number of different words in the corpus?
Explain your answer! 
(*1 point*)

In the unigram distribution the addition of new words leads to the additon of new probabilities for each of these words. This happens because the probability of each word is intependent from the other words.\
In the case of bigram and trigram the addition of new words may or may not introduce new combination. In particular, if the new words are conjuncted to already existing words, new probabilities are added to the distribution. On the contrary, if these new words appear in isolation, we won't add any probability.

### Task 2

a) Implement a function `sample(distribution)` for drawing a sample from the distributions $P (w)$, $P (w_i|w_{i−1})$
and $P (w_i|w_{i−1}, w_{i−2})$ according to the algorithm presented in the lecture. Make use of your solution of Task 1.\
(*3 points*)

In [38]:
import random

In [39]:
probs=unigram_distribution(words) # 

In [35]:
def sample(distribution, words, w1=None , w2=None , probs=None):
   # choose the distribution from which to sample 
    if distribution == 'unigram':
        p=unigram_distribution(words)
    
    elif distribution == 'bigram':
        p=bigram_distribution(w1, words, probs)
    else:
        p=trigram_distribution(w1, w2, words, probs)

    # dictionaries are unordered but this is a little trick to sort them by probabilities in reverse order
    sorted_p = dict(sorted(p.items(), key=lambda item: item[1], reverse=True))

    x=random.random()
    values= list(sorted_p.values()) # sorted probabilities  

    sump = values[0] # cumulative sum starting with the highest probability 

    # with this for loop we are summing one probability at a time and cheking if the sum exceeds x.
    for i in range(0,len(values)) :

        if sump - x < 0:
            sump+=values[i+1]
        else: 
            break 

    # when the break condition happens we check for the corresponding key. It may happen that more keys has the same probability
    # in this case we return the first one since we don't care about if one or the other if first
    for key, value in sorted_p.items():
        if value == values[i]:
            break

    return key 

    



b) Use the statistical information of the provided corpus to implement three different sentence generators, i.e., use
the distributions $P (w)$, $P (w_i|w_{i−1})$, $P (w_i|w_{i−1}, w_{i−2})$ and your code of a) to successively generate single words.
In other words, your first sentence generator should use the distribution $P (w)$, the second one $P (w_i|w_{i−1}) $ and the
third one $P (w_i|w_{i−1}, w_{i−2})$. The sentence should be returned as a string. Use the following naming conventions:
- `generate_sentence_unigram()`
- `generate_sentence_bigram()`
- `generate_sentence_trigram()`

In [36]:
def generate_sentence_unigram(words):
    # this code exactly mimics the pseudo code of the slides 
    eos=False
    sentence='' # initialize the sentence with an empty string 
    while not eos:
        w=sample('unigram', words) # sample from unigram 
        sentence+=w+ ' ' # add the word 
        if w=='.': # stop if a dot is sampled 
            eos=True
    
    return sentence


In [40]:
generate_sentence_unigram(words)

"identity is as the his is been short was traveling and on of an mort front satisfactory familiar -- spite branch on program they ghosts or produced bones the lindsey's time stop many of of them sharp of were the mere could they house a that as minneapolis . "

In [41]:
def generate_sentence_bigram(words, probs):
    
    eos=False
    w1 = sample('unigram', words)# get the starting word froma  unigram
    if w1=='.': # check the same stopping conditiona as in the sentence generated by the unigram
        eos=True

    while not eos:
        w=sample('bigram', words, w1, probs= probs )[-1] # get the last word in the tuple 
        sentence+= ' ' + w # add the word 
        if w=='.': # check the conditiona again 
            eos=True
        w1=w # the current word becomes the prevous one 
    
    return sentence
    

In [44]:
generate_sentence_bigram(words, probs) 

'was not already given to the world and different scanning techniques may be quickly as a central wrangled among other test predispositions destructive of the winter sky was obvious that only by lot of an important because of the peasants still every first contact a man to his work the af curve is always had a good thing to feel the wall street .'

In [45]:
def generate_sentence_trigram(words, probs):

    eos=False
    # sample w2 and w1 
    w2=sample('unigram', words) 
    w1=sample('bigram', words, w1=w2, probs=probs)[-1] # sample w1 using w2 as previous word and get only the last elemnt of the tuple

    if w2=='.': # same stopping condition if the first word is a dot 
        eos=True
        
    sentence=w2+' '+ w1+' ' # concatenate the two words  
    if w1=='.':
        eos=True

    while not eos:
        wi=sample('trigram',words, probs=probs, w1=w1, w2=w2)[-1] # sample from the trigram and get the last word of the triple 
        
        sentence+=wi+' ' # add the new word 
        
        if wi=='.': # check the condition again 
            eos=True

        w2=w1 # wi-2 <-- wi-1
        w1=wi # wi-1 <-- wi 
        # and look for the new wi


    return sentence


In [48]:
generate_sentence_trigram(words,probs) 

'in their first three games the longhorns have had the ball and handed it back . '

c) Describe the results of the three sentence generators you implemented. Try to explain the results.\
(*1 point*)

The more we shift towards trigram generated phrases from the unigram ones, we can observe the following:
- Phrases are shorter but more meaningful. 
- The algorithm becomes even more sensitive to initial choice of words, leading to a time increse of sentence generation (i.e. bad starting point can lead to very high waiting time).
